In [1]:
import numpy as np # used for arrays & loading data
import tensorflow as tf # arrays & loading data
from tensorflow.keras.models import Sequential  # model type that we will use
from tensorflow.keras.layers import Dense # we will use Dense layers
from tensorflow.keras.activations import linear, relu, sigmoid # some activation functions that we may use
from sklearn.preprocessing import StandardScaler # z-score normalization 

# suppress warnings
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

In [2]:
# loading all the training data
train_data = np.loadtxt('train.csv', delimiter=',', skiprows=1 )
print(train_data.shape)

(17996, 15)


In [3]:
X_train = train_data[:,:-1] # forming the input and output of the training data
y_train = train_data[:,-1]

y_train = np.expand_dims(y_train, axis=1) # make y 2D - the commands later will require it

print(X_train.shape)
print(y_train.shape)



# applying z-score to all the training data - make it compact for the algorithms to work better
standard_scaler = StandardScaler()
X_train_scaled = standard_scaler.fit_transform(X_train)

(17996, 14)
(17996, 1)


In [4]:
# we will use a Sequential model with Dense layers

model= Sequential(
    [
        tf.keras.Input(shape=(14,)), # input size (each song has 14 features)
        Dense(12,activation="relu", name="layer2"),   # usually, for multiclassification we use relu for all layers
        Dense(11,activation="linear", name="layer3"), # but for the last layer we use linear 
    ], name="multiclass_model"
)

In [5]:
# see details about the parameters and output of activation at every layer 
model.summary()

Model: "multiclass_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ layer2 (Dense)                       │ (None, 12)                  │             180 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer3 (Dense)                       │ (None, 11)                  │             143 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 323 (1.26 KB)

 Trainable params: 323 (1.26 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# define loss and optimizer of the Adam's algorithm
model.compile(
    # this is similar to gradient descent, but it is a much improved version
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # multiclass loss
    optimizer=tf.keras.optimizers.Adam(0.001), # preimplemented optimizer
)

In [7]:
# train the model "epochs" times
model.fit(
    X_train_scaled,y_train,
    epochs=40,
)

Epoch 1/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 617us/step - loss: 2.1607
Epoch 2/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.5981
Epoch 3/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.4535
Epoch 4/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.3968
Epoch 5/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.3736
Epoch 6/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.3528
Epoch 7/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.3527
Epoch 8/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step - loss: 1.3305
Epoch 9/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - loss: 1.3254
Epoch 10/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - loss: 1.3166
Epoch 11/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.3197
Epoch 12/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step - loss: 1.3277
Epoch 13/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.3173
Epoch 14/40
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1.3100
Epoch 15/40
563/563 ━━━━━━━━━

In [8]:
# loading all the test data
test_data = np.loadtxt('test.csv', delimiter=',', skiprows=1 )
print(test_data.shape)

(7713, 14)


In [9]:
X_cv = test_data[:3857,:] # forming the cv data
X_test = test_data[3857:,:] # forming the test data

print(X_cv.shape)
print(X_test.shape)


# applying z-score to all the cv & test data - make it compact for the algorithms to work better
X_cv_scaled = standard_scaler.transform(X_cv)
X_test_scaled = standard_scaler.transform(X_test)


(3857, 14)
(3856, 14)


In [11]:
y_prediction = model.predict(X_cv_scaled)  # prediction on cv dataset
print(y_prediction.shape)

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step
(3857, 11)


In [37]:
y_prediction_classes=np.empty((1,1))
print(y_prediction_classes)
print(y_prediction_classes.shape)
for i in range(y_prediction.shape[0]):
    max_element_index = np.argmax(y_prediction[i])
    y_prediction_classes = np.concatenate((y_prediction_classes,[[max_element_index]]),axis=0)
print(y_prediction_classes)

[[5.e-323]]
(1, 1)
[[5.e-323]
 [1.e+001]
 [8.e+000]
 ...
 [5.e+000]
 [1.e+001]
 [1.e+001]]
